# Score biweekly ML forecasts against re-calibrated ECWMF forecast with RPSS

Goal:

- Score biweekly ML forecasts against re-calibrated ECWMF forecast with RPSS

Requirements:

- [`xskillscore`](https://github.com/xarray-contrib/xskillscore)
- [renku datasets](https://renku-python.readthedocs.io/en/latest/commands.html#module-renku.cli.dataset) / file
    - observations
        - probabilistic:
            - renku dataset: `forecast-like-observations_2020_biweekly_terciled`
    - ML forecasts
        - probabilistic:
            - file: `../submissions/ML_prediction_2020.nc`
    - benchmark:
        - probabilistic:
            - renku dataset: `ecmwf_recalibrated_benchmark_2020_biweekly_terciled` (missing)
        
Output:
- RPSS score

In [1]:
# use most recent version of climetlab-s2s-ai-challenge
#!pip install git+https://github.com/ecmwf-lab/climetlab-s2s-ai-challenge.git

In [2]:
import matplotlib.pyplot as plt
import xarray as xr
import xskillscore as xs
import pandas as pd
import numpy as np

xr.set_options(keep_attrs=True)
xr.set_options(display_style='text')

cache_path = "../data"

# Get categorized

## 2020 observations

In [3]:
# to use retrieve from git lfs
#!renku storage pull ../data/forecast-like-observations_2020_biweekly_terciled.nc

In [4]:
obs_p = xr.open_dataset(f'{cache_path}/forecast-like-observations_2020_biweekly_terciled.nc')

obs_p.sizes

Frozen(SortedKeysDict({'category': 3, 'lead_time': 2, 'forecast_time': 53, 'latitude': 121, 'longitude': 240}))

## 2020 ML-forecasts forecasts

In [ ]:
#!renku storage pull ../submissions/ML_prediction_2020.nc

In [5]:
# submission for your ML model
fct_p = xr.open_dataset(f"../submissions/ML_prediction_2020.nc")
fct_p.sizes

Frozen(SortedKeysDict({'category': 3, 'lead_time': 2, 'forecast_time': 53, 'latitude': 121, 'longitude': 240}))

## ECMWF recalibrated benchmark

In [6]:
import climetlab as cml

# todo: use from renku dataset
# missing: 2 variables, 2 lead_time
bench_p = cml.load_dataset("s2s-ai-challenge-test-output-benchmark",
                         parameter='t2p', 
                         weeks='34'
                       ).to_xarray()

bench_p = bench_p.rename({'realization':'category', 't2p':'t2m'}) / 100

/work/mh0727/m300524/conda-envs/s2s-ai/lib/python3.7/site-packages/climetlab/plotting/drivers/magics/actions.py:36: UserWarning: Magics library could not be found
  warnings.warn(str(e))


By downloading data from this dataset, you agree to the terms and conditions defined at https://apps.ecmwf.int/datasets/data/s2s/licence/. If you do not agree with such terms, do not download the data. 


### checks

In [7]:
assert set(fct_p.dims) == set(obs_p.dims)
#assert set(bench_p.dims) == set(obs_p.dims)

In [8]:
assert fct_p.sizes == obs_p.sizes
#assert bench_p.sizes == obs_p.sizes

In [9]:
for v in obs_p.data_vars:
    assert v in fct_p.data_vars
#for v in obs_p.data_vars:
#    assert v in bench_p.data_vars

# RPS

## RPS ML model

In [10]:
rps_ML = xs.rps(obs_p, fct_p, category_edges=None, dim=['forecast_time'], input_distributions='p').compute()

In [11]:
#for v in rps_ML.data_vars:
#    rps_ML[v].plot(robust=True, col='lead_time')

## RPS re-calibrated ECMWF benchmark

In [12]:
rps_bench = xs.rps(obs_p, bench_p, category_edges=None, dim=['forecast_time'], input_distributions='p')
# rps_bench = rps_bench.where(mask)

could not mask all NaNs properly due to ValueError: indexes along dimension 'lead_time' are not equal


In [13]:
#rps_bench.t2m.plot(robust=True)

## RPSS

In [14]:
rpss = (1 - rps_ML/rps_bench)
# rpss['t2m'].squeeze().plot(vmin=-1,vmax=1,cmap='RdBu_r')

# scoring for RPS leaderboard

In [15]:
# check for -inf grid cells
if (rpss==-np.inf).to_array().any():
    (rpss == rpss.min()).sum()

    # dirty fix
    rpss = rpss.clip(-1, 1)

In [16]:
# what to do with requested grid cells where NaN is submitted? also penalize

In [17]:
mask = xr.ones_like(rpss.isel(lead_time=0,drop=True)).reset_coords(drop=True).t2m
boundary_tropics = 30
mask = xr.concat([mask.where(mask.latitude > boundary_tropics),
                mask.where(np.abs(mask.latitude) <= boundary_tropics),
                mask.where((mask.latitude < -boundary_tropics) & (mask.latitude > -60))],'area')
mask = mask.assign_coords(area=['northern_extratropics', 'tropics', 'southern_extratropics'])
mask.name='area'

mask = mask.where(rpss.t2m.isel(lead_time=0,drop=True).notnull())

In [18]:
# weighted area mean
weights = np.cos(np.deg2rad(np.abs(mask.latitude)))
scores = (rpss*mask).weighted(weights).mean('latitude').mean('longitude')
pd_scores = scores.reset_coords(drop=True).to_dataframe().unstack(0).round(2)

In [19]:
# final score
scores = rpss.weighted(weights).mean('latitude').mean('longitude')
# spatially weighted score averaged over lead_times and variables to one single value

In [20]:
# score transfered to leaderboard
scores = scores.to_array().mean().reset_coords(drop=True)
print(scores)

<xarray.DataArray ()>
array(-1.06167192)
